In [1]:
import os 
from pathlib import Path

import pandas as pd
import numpy as np

In [2]:
from sbscorer.sblegos.TransactionAnalyser import TransactionAnalyser as txa

In [3]:
FOLDER_NAME = 'gr18'
CHAIN = 'optimism'
extract_all = True

# Set path to data folder
current_dir = Path(os.getcwd())
src_dir = current_dir.parent.parent
PATH_TO_EXPORT = os.path.join(src_dir, 'tx_data', FOLDER_NAME)
PATH_TO_GR18 = os.path.join(src_dir, FOLDER_NAME)
voter_tx_file_name = 'gg18_votes.csv'
passport_file_name = 'gg18_voter_passports.csv'

tx_file_name = 'gr_18_optimism_all_tx.csv'

In [4]:
votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))
passports = pd.read_csv(os.path.join(PATH_TO_GR18, passport_file_name))

C:\Users\DELL Admin\AppData\Local\Temp\ipykernel_27076\1560736169.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  votes = pd.read_csv(os.path.join(PATH_TO_GR18, voter_tx_file_name))


In [5]:
round_names = votes['round_name'].unique()
round_names

array(['Climate Round', 'Web3 Open Source Software',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18', 'Web3 Social',
       'Zuzalu Continuous Innovation', 'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ', 'Token Engineering',
       'Ethereum Infrastructure', 'Arbitrum Domain Round'], dtype=object)

In [6]:
round_names =['Climate Round',
       'Web3 Community and Education ',
       'Global Chinese Community for Public Goods - GR18',
        'Web3 Social',
       'Zuzalu Continuous Innovation',
        'Meta Pool LatAm GG18',
       'ReFi DAO Local Node ',
        'Token Engineering', 'Web3 Open Source Software']
# round_names =['Web3 Open Source Software']

In [44]:
# create a dict with name of round as key and df_feature as value

dict_round_df = {}
for round_name in round_names:
    path_feature_file_new = os.path.join(PATH_TO_EXPORT, f'features_{round_name}_new.csv')
    df_feature = pd.read_csv(path_feature_file_new)
    df_feature.loc[:, 'EOA'] = df_feature['EOA'].str.lower()
    dict_round_df[round_name] = df_feature
    print(f'Round name: {round_name}, unique_voters: {df_feature.shape[0]}')
    print('Less than 10 tx: ', df_feature['less_10_tx'].sum())
    print("has_lcs: ", df_feature['has_lcs'].sum())
    print("\n")

Round name: Climate Round, unique_voters: 5610
Less than 10 tx:  1690
has_lcs:  414


Round name: Web3 Community and Education , unique_voters: 12413
Less than 10 tx:  2629
has_lcs:  727


Round name: Global Chinese Community for Public Goods - GR18, unique_voters: 545
Less than 10 tx:  143
has_lcs:  34


Round name: Web3 Social, unique_voters: 3828
Less than 10 tx:  639
has_lcs:  114


Round name: Zuzalu Continuous Innovation, unique_voters: 695
Less than 10 tx:  281
has_lcs:  40


Round name: Meta Pool LatAm GG18, unique_voters: 610
Less than 10 tx:  264
has_lcs:  50


Round name: ReFi DAO Local Node , unique_voters: 752
Less than 10 tx:  397
has_lcs:  56


Round name: Token Engineering, unique_voters: 1830
Less than 10 tx:  830
has_lcs:  193


Round name: Web3 Open Source Software, unique_voters: 39694
Less than 10 tx:  7974
has_lcs:  3100




In [45]:
votes.columns

Index(['round_name', 'project_name', 'transaction', 'blockNumber', 'projectId',
       'roundId', 'voter', 'grantAddress', 'token', 'amount', 'amountUSD',
       'amountRoundToken', 'score', 'scoreTimestamp', 'tx_timestamp'],
      dtype='object')

In [46]:
passports.columns

Index(['voter', 'score'], dtype='object')

In [47]:
df_feature.columns

Index(['EOA', 'count_tx', 'less_10_tx', 'count_same_seed',
       'count_same_seed_naive', 'same_seed', 'same_seed_naive',
       'seed_suspicious', 'count_interact_other_ctbt', 'first_in_tx_from',
       'first_in_tx_gas_limit', 'first_in_tx_gas_used',
       'first_in_tx_eth_value', 'first_in_tx_timestamp', 'to_address',
       'first_out_tx_gas_limit', 'first_out_tx_gas_used',
       'first_out_tx_eth_value', 'first_out_tx_timestamp', 'lcs',
       'cluster_size_lcs', 'mean_score_lcs', 'max_score_lcs', 'has_lcs'],
      dtype='object')

In [48]:
# merge votes and passport 
df_vp = votes.merge(passports, how='left', left_on='voter', right_on='voter')

In [97]:
for round_name in round_names:
    df_vpr = df_vp[df_vp['round_name'] == round_name]
    df_vpr.loc[:, 'voter'] = df_vpr['voter'].str.lower()
    df_vprf = df_vpr.merge(dict_round_df[round_name], how='left', left_on='voter', right_on='EOA')
    df_vprf['has_interact_other_ctbt'] = df_vprf['count_interact_other_ctbt'] > 0
    df_vprf['has_passport'] = df_vprf['score_x'] > 20 

    df_sum = df_vprf.groupby('project_name').agg({'voter': 'count', 'has_passport': 'sum', 'less_10_tx': 'sum', 'seed_suspicious':'sum', 'has_interact_other_ctbt': 'sum', 'has_lcs': 'sum'}).sort_values(by='has_lcs', ascending=False)
    df_sum['per_lcs'] = df_sum['has_lcs'] / df_sum['voter']

    print(f'Round name: {round_name}, unique_voters: {df_vprf.shape[0]}')
    print(df_sum.sort_values(by='per_lcs', ascending=False).head(20))


Round name: Climate Round, unique_voters: 14002
                                                    voter  has_passport   
project_name                                                              
Litter Token                                          990           881  \
Vanara                                                 60            42   
Barichara Regeneration Fund – Prototyping a Bio...    116            88   
SANE Foundation: Pioneering Web3 Climate Soluti...     37            23   
Roela                                                  52            29   
Traditional Dream Factory                              86            56   
AI For Trees                                           55            39   
Leaf Track                                             63            43   
Sustainable Heating with Compute Power                 93            66   
Shamba Network dMRV                                   144           100   
AYOWECCA UGANDA:      Training rural farmers & ...  